In [45]:
# Biomedical Machine Learning - DAT Binding Dataset Analysis
# Processing pipeline for molecular descriptors, PCA, and modeling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")


In [46]:
# Step 1: Load and merge the DAT binding dataset
print("=" * 70)
print("LOADING DAT BINDING DATASET")
print("=" * 70)

print("Loading molecular SMILES and binding activity data...")

# Load SMILES data (pubdata.smi)
print("\n1. Loading SMILES data...")
smiles_data = pd.read_csv('dataset_DAT_binding_Ki/pubdata.smi', 
                         sep='\t', 
                         header=None, 
                         names=['smiles', 'chembl_id'])

print(f"SMILES file shape: {smiles_data.shape}")
print("First few SMILES entries:")
print(smiles_data.head())

# Load activity data (pubdata.act)
print("\n2. Loading pKi activity data...")
activity_data = pd.read_csv('dataset_DAT_binding_Ki/pubdata.act', 
                           sep='\t', 
                           header=None, 
                           names=['chembl_id', 'pKi'])

print(f"Activity file shape: {activity_data.shape}")
print("First few activity entries:")
print(activity_data.head())

# Check for data consistency
print(f"\n3. Data consistency check:")
print(f"Unique ChEMBL IDs in SMILES file: {smiles_data['chembl_id'].nunique()}")
print(f"Unique ChEMBL IDs in activity file: {activity_data['chembl_id'].nunique()}")
print(f"Overlapping ChEMBL IDs: {len(set(smiles_data['chembl_id']) & set(activity_data['chembl_id']))}")

# Merge the datasets on ChEMBL ID
print(f"\n4. Merging datasets on ChEMBL ID...")
data = pd.merge(smiles_data, activity_data, on='chembl_id', how='inner')

print(f"Merged dataset shape: {data.shape}")
print(f"Columns: {data.columns.tolist()}")

# Reorder columns for better organization
data = data[['chembl_id', 'pKi', 'smiles']]

# Create binary activity classification based on pKi threshold
# Active: pKi >= 6.5 (corresponds to Ki <= ~316 nM, a common medicinal chemistry threshold)
activity_threshold = 6.5
data['activity_class'] = (data['pKi'] >= activity_threshold).astype(int)

print(f"\n5. Activity classification (threshold: pKi >= {activity_threshold}):")
print(f"Active compounds (pKi >= {activity_threshold}): {(data['activity_class'] == 1).sum()}")
print(f"Inactive compounds (pKi < {activity_threshold}): {(data['activity_class'] == 0).sum()}")
print(f"Activity ratio: {(data['activity_class'] == 1).mean():.2%} active")

print("\nFinal dataset:")
print(data.head())

print("\nDataset summary:")
print(data.info())
print("\nMissing values:")
print(data.isnull().sum())

print(f"\npKi distribution:")
print(data['pKi'].describe())

print(f"\n✅ Successfully loaded and merged {data.shape[0]} compounds")
print(f"✅ Ready for molecular descriptor generation and PCA analysis")


LOADING DAT BINDING DATASET
Loading molecular SMILES and binding activity data...

1. Loading SMILES data...
SMILES file shape: (541, 2)
First few SMILES entries:
                                         smiles    chembl_id
0       CN(C)CCCC1(c2ccc(F)cc2)OCc2cc(C#N)ccc21    CHEMBL549
1                 CCOC(=O)C1(c2ccccc2)CCN(C)CC1    CHEMBL607
2                    COC(=O)C(c1ccccc1)C1CCCCN1    CHEMBL904
3          Fc1ccc(C2CCNCC2COc2ccc3c(c2)OCO3)cc1   CHEMBL1708
4  CCCCN1C2CCC1CC(OC(c1ccc(F)cc1)c1ccc(F)cc1)C2  CHEMBL11493

2. Loading pKi activity data...
Activity file shape: (541, 2)
First few activity entries:
     chembl_id   pKi
0    CHEMBL549  4.78
1    CHEMBL607  4.75
2    CHEMBL904  7.13
3   CHEMBL1708  7.01
4  CHEMBL11493  7.98

3. Data consistency check:
Unique ChEMBL IDs in SMILES file: 541
Unique ChEMBL IDs in activity file: 541
Overlapping ChEMBL IDs: 541

4. Merging datasets on ChEMBL ID...
Merged dataset shape: (541, 3)
Columns: ['smiles', 'chembl_id', 'pKi']

5. Activity

In [60]:
# Step 2: Dataset Overview and Exploratory Analysis
print("=" * 70)
print("DATASET OVERVIEW AND INITIAL EXPLORATION")
print("=" * 70)

print("Analyzing the DAT binding dataset...")

# Basic dataset statistics
print(f"\nDATASET STATISTICS:")
print(f"Total compounds: {data.shape[0]}")
print(f"Features: ChEMBL ID, pKi (binding affinity), SMILES (molecular structure)")

# pKi distribution analysis
print(f"\nBINDING AFFINITY (pKi) ANALYSIS:")
print(f"pKi range: {data['pKi'].min():.2f} - {data['pKi'].max():.2f}")
print(f"Mean pKi: {data['pKi'].mean():.2f} ± {data['pKi'].std():.2f}")
print(f"Median pKi: {data['pKi'].median():.2f}")

# Activity classification breakdown
active_count = (data['activity_class'] == 1).sum()
inactive_count = (data['activity_class'] == 0).sum()

print(f"\nACTIVITY CLASSIFICATION:")
print(f"Active compounds (pKi ≥ 6.5):   {active_count:3d} ({100*active_count/len(data):.1f}%)")
print(f"Inactive compounds (pKi < 6.5): {inactive_count:3d} ({100*inactive_count/len(data):.1f}%)")

# SMILES length analysis (indicator of molecular complexity)
data['smiles_length'] = data['smiles'].str.len()
print(f"\nMOLECULAR COMPLEXITY (SMILES length):")
print(f"Average SMILES length: {data['smiles_length'].mean():.1f} characters")
print(f"Range: {data['smiles_length'].min()} - {data['smiles_length'].max()} characters")

# Show examples from different pKi ranges
print(f"\nEXAMPLE COMPOUNDS:")
print("High activity (pKi > 8.0):")
high_activity = data[data['pKi'] > 8.0].head(2)
for _, row in high_activity.iterrows():
    print(f"  {row['chembl_id']}: pKi={row['pKi']:.2f}, SMILES={row['smiles'][:60]}...")

print("Low activity (pKi < 5.0):")
low_activity = data[data['pKi'] < 5.0].head(2)
for _, row in low_activity.iterrows():
    print(f"  {row['chembl_id']}: pKi={row['pKi']:.2f}, SMILES={row['smiles'][:60]}...")

# Clean up temporary column
data = data.drop('smiles_length', axis=1)

print(f"\n✅ Dataset overview complete - ready for molecular descriptor generation!")


DATASET OVERVIEW AND INITIAL EXPLORATION
Analyzing the DAT binding dataset...

DATASET STATISTICS:
Total compounds: 541
Features: ChEMBL ID, pKi (binding affinity), SMILES (molecular structure)

BINDING AFFINITY (pKi) ANALYSIS:
pKi range: 3.41 - 10.40
Mean pKi: 6.92 ± 1.17
Median pKi: 6.95

ACTIVITY CLASSIFICATION:
Active compounds (pKi ≥ 6.5):   334 (61.7%)
Inactive compounds (pKi < 6.5): 207 (38.3%)

MOLECULAR COMPLEXITY (SMILES length):
Average SMILES length: 43.2 characters
Range: 18 - 75 characters

EXAMPLE COMPOUNDS:
High activity (pKi > 8.0):
  CHEMBL27336: pKi=8.66, SMILES=CCC1C(c2ccc(Cl)cc2)CC2CCC1N2C...
  CHEMBL28394: pKi=9.38, SMILES=CN1C2CCC1C(C=CCl)C(c1ccc(Cl)cc1)C2...
Low activity (pKi < 5.0):
  CHEMBL549: pKi=4.78, SMILES=CN(C)CCCC1(c2ccc(F)cc2)OCc2cc(C#N)ccc21...
  CHEMBL607: pKi=4.75, SMILES=CCOC(=O)C1(c2ccccc2)CCN(C)CC1...

✅ Dataset overview complete - ready for molecular descriptor generation!


In [48]:
# Step 3: Data Cleaning - Validate and prepare data for analysis
print("=" * 70)
print("STEP 3: DATA CLEANING AND VALIDATION")
print("=" * 70)

def validate_smiles(smiles):
    """Check if SMILES string is valid using RDKit"""
    if pd.isna(smiles) or smiles == '':
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

# Remove rows with missing pKi values
print(f"Rows before removing missing pKi: {len(data)}")
data_clean = data.dropna(subset=['pKi']).copy()
print(f"Rows after removing missing pKi: {len(data_clean)}")

# Remove rows with invalid SMILES
print(f"Validating SMILES...")
valid_smiles = data_clean['smiles'].apply(validate_smiles)
print(f"Valid SMILES: {valid_smiles.sum()} / {len(data_clean)}")

data_clean = data_clean[valid_smiles].copy()
print(f"Rows after removing invalid SMILES: {len(data_clean)}")

# Check for exact duplicates
duplicates = data_clean.duplicated()
print(f"Exact duplicates found: {duplicates.sum()}")
if duplicates.sum() > 0:
    data_clean = data_clean[~duplicates].copy()
    print(f"Rows after removing exact duplicates: {len(data_clean)}")

print(f"\nFinal cleaned dataset shape: {data_clean.shape}")


STEP 3: DATA CLEANING AND VALIDATION
Rows before removing missing pKi: 541
Rows after removing missing pKi: 541
Validating SMILES...
Valid SMILES: 541 / 541
Rows after removing invalid SMILES: 541
Exact duplicates found: 0

Final cleaned dataset shape: (541, 4)


In [49]:
# Step 3: Handle duplicate SMILES by averaging pKi values
print("Checking for duplicate SMILES...")

# Group by SMILES and check for duplicates
smiles_counts = data_clean['smiles'].value_counts()
duplicate_smiles = smiles_counts[smiles_counts > 1]
print(f"Number of unique SMILES with duplicates: {len(duplicate_smiles)}")

if len(duplicate_smiles) > 0:
    print(f"Total duplicate entries: {duplicate_smiles.sum() - len(duplicate_smiles)}")
    
    # Show some examples of duplicates
    print("\nExamples of duplicate SMILES:")
    for smiles in duplicate_smiles.head(3).index:
        subset = data_clean[data_clean['smiles'] == smiles]
        print(f"SMILES: {smiles[:50]}...")
        print(f"  Entries: {len(subset)}, pKi values: {subset['pKi'].tolist()}")
    
    # Collapse duplicates by averaging pKi values
    print("\nCollapsing duplicates by averaging pKi values...")
    data_collapsed = data_clean.groupby('smiles').agg({
        'chembl_id': 'first',  # Keep first ChEMBL ID
        'pKi': 'mean',  # Average pKi values
        'activity_class': lambda x: 1 if x.mean() >= 0.5 else 0  # Majority class
    }).reset_index()
    
    print(f"Shape after collapsing duplicates: {data_collapsed.shape}")
    
    # Reorder columns
    data_collapsed = data_collapsed[['chembl_id', 'pKi', 'activity_class', 'smiles']]
else:
    print("No duplicate SMILES found.")
    data_collapsed = data_clean.copy()

print(f"\nFinal dataset for analysis: {data_collapsed.shape}")
print("\npKi distribution:")
print(data_collapsed['pKi'].describe())


Checking for duplicate SMILES...
Number of unique SMILES with duplicates: 0
No duplicate SMILES found.

Final dataset for analysis: (541, 4)

pKi distribution:
count    541.000000
mean       6.917246
std        1.169415
min        3.410000
25%        5.960000
50%        6.950000
75%        7.800000
max       10.400000
Name: pKi, dtype: float64
